# TAREA 9

##### Esthefany Ortiz
#### 148571
#### El objetivo de la tarea es utilizar Dask, una libreria de python diseñada para cómputo en gran escala.

In [1]:
### importamos librerias que quizas usemos 

from dask.distributed import Client
from dask import delayed
import pandas as pd
client = Client("scheduler:8786")
import time
from sklearn.base import TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

In [2]:
from dask import dataframe
trips_df = dataframe.read_csv("/data/trips.csv")
#Utilizamos el truco para convertir a datetime.
trips_df.tpep_pickup_datetime = trips_df.tpep_pickup_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime = trips_df.tpep_dropoff_datetime.astype('M8[us]')

In [3]:
#agrega una columna con el porcentaje de propina
trips_df["tip_perc"] = trips_df.tip_amount/trips_df.fare_amount
# Agrega una columna para la duración del viaje
trips_df["trip_duration"] = trips_df.tpep_dropoff_datetime - trips_df.tpep_pickup_datetime
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,tip_perc,trip_duration
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,0.209091,00:19:30
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,0.000000,00:09:47
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,0.133333,00:10:27
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,0.117647,00:09:11
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,0.221333,00:09:28


In [4]:
# quitamos los NAs y quitamos los viajes que tienen un costo de cero. 
trips_df = trips_df[trips_df.fare_amount > 0]
trips_df = trips_df.dropna()

In [ ]:
trips_df.describe()

#### Hacemos el primer ejercicio utillizando Sklearn y hacemos una regresión lineal

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model 

In [6]:
params = {'max_iter' :[50, 100,200],'alpha':[0.001,0.01,0.1,1]}
reg = linear_model.LinearRegression()
modelo = GridSearchCV(reg, params, cv=10, scoring='neg_mean_squared_error',verbose=1, n_jobs=-1)


In [7]:
X= trips_df.drop(['tpep_dropoff_datetime', 'tip_perc','tpep_pickup_datetime', 'tip_amount', 'trip_duration','car_type'], axis=1).compute()
y = trips_df['tip_perc'].compute()


In [ ]:
X.head()

In [8]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
delayed_reg = delayed(reg)

In [9]:
%%time
res = delayed_reg.fit(X, y).compute()

CPU times: user 10 ms, sys: 10 ms, total: 20 ms
Wall time: 216 ms


In [26]:
res.coef_

array([-4.21563716e-04, -1.02494275e-03,  3.07645252e-06,  1.25563688e-03])

#### Ahora hacemos el mismo ejercicio utilizando DaskML y comparamos resultados


In [11]:
from dask_searchcv import GridSearchCV as GSCVDask
from dask_ml.linear_model import LinearRegression

In [ ]:
params = {'max_iter' :[50, 100,200],
          'C':[0.001,0.01,0.1,1]}

In [12]:
reg_dask = GSCVDask(LinearRegression(), params, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

In [14]:
%%time
delayed_dask=delayed(reg_dask)
res_dask=delayed_dask.fit(X,y)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.6 ms


El modelo de Dask fue mucho más lento que utilizando Sklearon

In [25]:
res_dask.coef_

Delayed('getattr-63f62bffa454d6328361a70e6c0c1bb9')

In [ ]:
El resultado más rapido lo obtuvimos utilizando las funciones de Dask. Saludos.